In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense



# Load the Bitcoin price data
bitcoin_data = pd.read_csv('/content/BTC-2021min.csv')

# Remove unnecessary columns (if any)
bitcoin_data = bitcoin_data[['date', 'close']]

# Sort the data by date
bitcoin_data = bitcoin_data.sort_values('date')

# Convert the 'Date' column to datetime
bitcoin_data['date'] = pd.to_datetime(bitcoin_data['date'])

# Set the 'Date' column as the index
bitcoin_data.set_index('date', inplace=True)

# Resample the data to daily frequency (if needed)
bitcoin_data = bitcoin_data.resample('D').last()

# Fill missing values with previous values (forward fill)
bitcoin_data.fillna(method='ffill', inplace=True)

# Normalize the 'Close' prices to be between 0 and 1
scaler = MinMaxScaler()
bitcoin_data['close'] = scaler.fit_transform(bitcoin_data['close'].values.reshape(-1, 1))

# Split the data into input sequences (X) and target sequences (y)
input_seq_length = 10  # Number of past prices to use as input sequence
X, y = [], []
for i in range(len(bitcoin_data) - input_seq_length):
    X.append(bitcoin_data['close'].values[i:i+input_seq_length])
    y.append(bitcoin_data['close'].values[i+input_seq_length])
X = np.array(X)
y = np.array(y)

# Split the data into training, validation, and test sets
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Reshape the input sequences to be 3D arrays (batch_size, timesteps, input_dim)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))




In [8]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
# Define the sequence-to-sequence model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(input_seq_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
9/9 [==============================] - 2s 48ms/step - loss: 0.2450 - val_loss: 0.2082
Epoch 2/10
9/9 [==============================] - 0s 14ms/step - loss: 0.1360 - val_loss: 0.0901
Epoch 3/10
9/9 [==============================] - 0s 14ms/step - loss: 0.0434 - val_loss: 0.0109
Epoch 4/10
9/9 [==============================] - 0s 12ms/step - loss: 0.0238 - val_loss: 0.0145
Epoch 5/10
9/9 [==============================] - 0s 14ms/step - loss: 0.0141 - val_loss: 0.0144
Epoch 6/10
9/9 [==============================] - 0s 12ms/step - loss: 0.0151 - val_loss: 0.0109
Epoch 7/10
9/9 [==============================] - 0s 12ms/step - loss: 0.0121 - val_loss: 0.0096
Epoch 8/10
9/9 [==============================] - 0s 12ms/step - loss: 0.0109 - val_loss: 0.0085
Epoch 9/10
9/9 [==============================] - 0s 12ms/step - loss: 0.0098 - val_loss: 0.0082
Epoch 10/10
9/9 [==============================] - 0s 14ms/step - loss: 0.0090 - val_loss: 0.0080


In [34]:
# Generate predictions for the test set
y_pred_test = model.predict(X_test)

# Evaluate the model using appropriate evaluation metrics
# For example, calculate Mean Squared Error (MSE)
from sklearn.metrics import mean_squared_error
mse = np.mean(np.square(y_test - y_pred_test))
# Set a maximum threshold for MSE

max_mse = 4.0  # Example threshold value

# Clip MSE values to the maximum threshold
mse = np.clip(mse, None, max_mse)

# Calculate accuracy as a percentage
accuracy = 100 * (1 - mse / max_mse)
print("Original MSE:", mse)
print("Accuracy (%):", accuracy)

2/2 [==============================] - 0s 11ms/step
Original MSE: 0.11555513452989627
Accuracy (%): 97.1111216367526
